# Project_One_이상봉

## 주제와 목적
최근 북미, 유럽, 일본, 그외 지역에서의 게임 판매량 데이터를 분석하고  
분석을 바탕으로 다음 분기에 게임 설계 방향을 정하는것이다    
정확한 분석을 위해서는 게임의 판매량에 영향을 미치는요인이 무엇인지 알아내야한다.  

## 우선 분석과정을 4개의 작은 단위로 나눠볼 수 있다
1. 지역별 선호하는 장르가 다른가?
2. 연도별 트랜드가 있는가?
3. 플랫폼에따른 판매량의 차이가 있는가?
4. 판매량이 가장 높은 게임들은 무슨 특징을 가지고 있는가?

이 분석과정을 통해, **"잘 팔리는"** 게임을 만들기위해 어떤 부분을 우선시 해야하는지를 판단해야 한다


분석을 시작하기전에, 데이터를 좀더 깔끔하게 다듬는 전처리 작업을 진행하였다  
# 전처리작업(EDA)
우선 데이터내의 수치를 일관성있게 통일하기위해 전처리작업을 진행하였다  
전처리작업이란, 숫자의단위나 알파벳의 대소문자의 통일, 알수없는값이나 비어있는값을 처리하여 데이터의 오류를 줄이는 과정이다.

In [137]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, f_oneway


# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/AI_bootcamp/vgames2.csv', encoding='CP949')

# 인덱스제거
df = df.drop(['Unnamed: 0','Name','Publisher'], axis=1)

In [138]:
# 판매량을 (0.00) 단위로 통일하고 float으로 변경
def unification(string):
    # K가 있다면 지우고 천 곱하고 백만으로 나누기
    if ('K' in string) == True:
        # 만약 0K 라면 0.001로 바로바꾸기
        if string == '0K':
            return float(0.001)
        else:
            remove_k = string.replace('K','')
            return float(remove_k)*0.001
    # M이 있다면 지우기
    elif ('M' in string) == True:
        # 만약 OM 이라면 1.0으로 바꾸기
        if string == '0M':
            return float(1.0)
        else:
            remove_m = string.replace('M','')
            return float(remove_m)
    # 둘다없다면 타입만바꾸기
    else:
        return float(string)

# 판매량 칼럽의 단위통일시키기
df['NA_Sales'] = df['NA_Sales'].apply(unification)
df['JP_Sales'] = df['JP_Sales'].apply(unification)
df['EU_Sales'] = df['EU_Sales'].apply(unification)
df['Other_Sales'] = df['Other_Sales'].apply(unification)

# datatype이 object->float으로 되었는지 확인
df.dtypes

Platform        object
Year           float64
Genre           object
NA_Sales       float64
EU_Sales       float64
JP_Sales       float64
Other_Sales    float64
dtype: object

In [139]:
# Year 칼럼 네자리로 통일시키기
def make_year(year):
    if year < 21:
        return float(year+2000)
    elif year > 80 and year <= 99:
        return float(year+1900)
    else:
        return float(year)

df['Year'] = df['Year'].apply(make_year)

In [140]:
# 결측치 위치 확인하기
# 이값은 후에 다시 다루기
df.isna().sum()

Platform         0
Year           271
Genre           50
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64

전처리가 끝난 데이터로 분석해보자  


# 1. 지역별 선호하는 장르가 다른가?  
다시말해  
**"장르의 지역별 판매량이 서로 비슷한가가?"**  
**"미국에서 잘 팔린 레이싱게임이 다른지역에서도 잘 팔리는가,  
아니면 미국에서만 잘팔렸는가?"**  
정도로 이해할 수 있다  

우선 장르들이 각각의 나라에서의 총판매량을 알아야한다  
그 후 각 장르의 총판매량이 얼마나 비슷한지를 확인하여
비슷하다면, 선호하는 장르가 비슷하다  
비슷하지않다면, 선호하는 장르다 다르다  
고 결론내릴 수 있다


In [141]:
# 각 지역에서 장르의 총 판매량 확인
df_genre_loc = df.groupby('Genre').sum().drop('Year',axis=1)
df_genre_loc

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Genre,,,,
Action,878.814,528.163,169.910,194.461
Adventure,108.660,70.052,53.954,19.815
Fighting,223.200,103.224,89.344,38.641
Misc,410.055,220.366,119.274,75.045
Platform,446.440,201.610,133.656,54.536
Puzzle,123.351,51.732,58.871,13.522
Racing,360.191,239.101,63.409,78.102
Role-Playing,323.542,187.583,344.955,61.965
Shooter,582.921,313.052,42.284,105.653


In [142]:
# H0 = 지역별 장르의 판매량 분포는 차이가없다

# 각 지역별 분포를 비교하는 F-test 실시
dff_genre_loc = f_oneway(df_genre_loc['EU_Sales'],df_genre_loc['JP_Sales'],df_genre_loc['NA_Sales'],df_genre_loc['Other_Sales'])
print(dff_genre_loc.pvalue)

# p-value 매우작음 -> H0 기각 -> 지역별 장르의 판매량 분포는 차이가 있다

# 즉 지역별로 선호하는 장르가 다르다

9.854620901886625e-05


# 연도별 게임의 트랜드가 있을까?

"연도별 가장많이 출시된 장르의 갯수가 다를까?"

 연도별 출시된 게임들의 장르 총 갯수를 구해야한다  

각 연도별 장르들의 총 판매량을 구한 후  
판매량의 분포차이를 비교해보자

In [164]:
pd.crosstab(df.Year,df.Genre)

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Year,,,,,,,,,,,,
1980.0,1,0,1,4,0,0,0,0,2,0,0,0
1981.0,25,0,0,0,3,2,1,0,10,1,4,0
1982.0,18,0,0,1,5,3,2,0,5,0,2,0
1983.0,7,1,0,1,5,1,0,0,1,0,1,0
1984.0,1,0,0,1,1,3,3,0,3,0,2,0
1985.0,2,0,1,0,4,4,0,0,1,1,1,0
1986.0,6,0,0,0,6,0,1,1,4,0,3,0
1987.0,2,1,2,0,2,0,0,3,2,0,4,0
1988.0,2,0,0,0,4,1,1,3,1,1,2,0


통계학적 방법을 이용하여 데이터를 **단순화** 시켜  
효율적이고 정확한 비교를 해야한다  

단순화란 무엇일까? 예를들면  
산에 조난되었는데 먹을게 버섯밖에없다  
독버섯은 여러가지 특징들이 있지만, 정확히 구분하기가 힘들다  
하지만 책에서 "독버섯의 95%가 빨간색이다" 라는 글이 생각났고 빨간버섯은 피했다    
이와 같은 맥락이다  
수많은 데이터가 있고 그중 가장 영향력있는 조건하나를 찾거나 조합해내서  
그 조건을 기준으로 최대한 정확하게 판단하는것이다  
